In [1]:
import bw2io as bi
import bw2data as bd
import sys
sys.path.append("/Users/akim/PycharmProjects/bigfood_db")

# Local files
from bigfood_db.import_databases import import_ecoinvent
from bigfood_db.importers import Agribalyse13Importer
from bigfood_db.utils import create_location_mapping_glo, create_location_mapping_rer, modify_exchanges


if __name__ == '__main__':
    project = "Food ei38"
    bd.projects.set_current(project)

    ei_path = "/Users/akim/Documents/LCA_files/ecoinvent_38_cutoff/datasets"
    ei_name = "ecoinvent 3.8 cutoff"

    bi.bw2setup()
    b3 = bd.Database('biosphere3')
    import_ecoinvent(ei_path, ei_name)
    ei = bd.Database(ei_name)

    ag_path = "/Users/akim/Documents/LCA_files/agribalyse_13/Agribalyse CSV FINAL_no links_Nov2016v3.CSV"
    ag_name = "Agribalyse 1.3"
    ag = Agribalyse13Importer(ag_path, ag_name)
#     ag.add_unlinked_activities()
#     mapping = create_location_mapping(ag, ei_name)
#     ag = modify_exchanges(ag, mapping, ei_name)
    

Biosphere database already present!!! No setup is needed
ecoinvent 3.8 cutoff database already present!!! No import is needed
Extracted 1189 unallocated datasets in 3.63 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: assign_only_product_as_production
Applying strategy: drop_unspecified_subcategories
Applying strategy: sp_allocate_products
Applying strategy: fix_zero_allocation_products
Applying strategy: split_simapro_name_geo
Applying strategy: strip_biosphere_exc_locations
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: set_code_by_activity_hash
Applying strategy: link_technosphere_based_on_name_unit_location
Applying strategy: change_electricity_unit_mj_to_kwh
Applying strategy: set_lognormal_loc_value_uncertainty_safe
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_simapro_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying s

In [2]:
ag.statistics()

1261 datasets
115259 exchanges
3294 unlinked exchanges
  Type biosphere: 440 unique unlinked exchanges
  Type technosphere: 77 unique unlinked exchanges


(1261, 115259, 3294)

# Technosphere

## 1. Allocation for activities with RER locations

In [3]:
mapping_rer = create_location_mapping_rer(ag, ei_name)

In [4]:
# Add nitric acid manually
nitric_acid_rer_wo_ru = [
    act for act in ei 
    if "nitric acid production" in act['name']
    and "RER w/o" in act['location']
    and "nitric acid, without water, in 50% solution state" in act['reference product']
]
assert len(nitric_acid_rer_wo_ru)==1
nitric_acid_ru = [
    act for act in ei 
    if "nitric acid production" in act['name']
    and "RU" == act['location']
    and "nitric acid, without water, in 50% solution state" in act['reference product']
]
assert len(nitric_acid_ru)==1
mapping_rer.append(
    {
        ("nitric acid production, product in 50% solution state", "RER"): [
            (ei.name, nitric_acid_rer_wo_ru[0]['code']),
            (ei.name, nitric_acid_ru[0]['code']),
        ]
    }
)
mapping_rer

[{('reinforcing steel production',
   'RER'): [('ecoinvent 3.8 cutoff',
    '98d4e36247aefadf9db37e9312c50a14'), ('ecoinvent 3.8 cutoff', '1c8e803b4a1eeb405ff8800532239116')]},
 {('sawnwood production, softwood, raw, dried (u=20%)',
   'RER'): [('ecoinvent 3.8 cutoff',
    '7e691769cd3546db199f7c3e66ed3760'), ('ecoinvent 3.8 cutoff', '773a438348a177aa8103249e1f82493c')]},
 {('nitric acid production, product in 50% solution state',
   'RER'): [('ecoinvent 3.8 cutoff',
    'e7b829f39d2c08681dd4b6b236fa3271'), ('ecoinvent 3.8 cutoff', '296b15dd033de76876de600383837ec1')]}]

In [5]:
ag = modify_exchanges(ag, mapping_rer, ei_name)
ag.match_database(ei_name, fields=['name', 'location', 'unit', 'reference product'])
# ag.match_database(ei_name, fields=['name', 'location', 'unit'])
ag.statistics()

Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
1261 datasets
115284 exchanges
3269 unlinked exchanges
  Type biosphere: 440 unique unlinked exchanges
  Type technosphere: 74 unique unlinked exchanges


(1261, 115284, 3269)

## 2. Allocation for activities with GLO locations

In [6]:
%%time
mapping_glo = create_location_mapping_glo(ag, ei_name)
mapping_glo

CPU times: user 44.1 s, sys: 12.9 s, total: 57.1 s
Wall time: 57.8 s


[{('market for transport, freight, lorry >32 metric ton, EURO4',
   'GLO'): [('ecoinvent 3.8 cutoff',
    '43b66a83281658133104438bea9040f0'), ('ecoinvent 3.8 cutoff', 'e80fa9b85960f64b02dea371bfdfc790')]},
 {('market for transport, freight, lorry, unspecified',
   'GLO'): [('ecoinvent 3.8 cutoff',
    '2b0c1793924e1bb3e821cbe15bee1a32'), ('ecoinvent 3.8 cutoff', '6edfee34a352760ad1029a3aa342e00a'), ('ecoinvent 3.8 cutoff',
    'f42f0f6bf959f03b728ef93398630236'), ('ecoinvent 3.8 cutoff',
    '9620867f9e5b65a1dc2a63e5eb67c40e')]},
 {('market for lime',
   'GLO'): [('ecoinvent 3.8 cutoff',
    '53ff56d23191ee5992141d88dd6c9f4f'), ('ecoinvent 3.8 cutoff', 'cb7908bc37fd0ee15ae23f425eabb796')]},
 {('market for iron sulfate',
   'GLO'): [('ecoinvent 3.8 cutoff',
    '6025ff1e208cc532654b15bc75f64d78'), ('ecoinvent 3.8 cutoff', '75caf350f8898028dd5f1692ac67f586')]},
 {('market for transport, freight, inland waterways, barge',
   'GLO'): [('ecoinvent 3.8 cutoff',
    'e82a2f2f1f59463b25cf56e3

In [7]:
ag = modify_exchanges(ag, mapping_glo, ei_name)
ag.match_database(ei_name, fields=['name', 'location', 'unit'])

ag.statistics()

Applying strategy: link_iterable_by_fields
1261 datasets
116027 exchanges
2903 unlinked exchanges
  Type biosphere: 440 unique unlinked exchanges
  Type technosphere: 40 unique unlinked exchanges


(1261, 116027, 2903)

## 3. Still unlinked

In [8]:
[(act['name'], act.get('location', None)) for act in ag.unlinked if "tech" in act['type']]

[('market for copper', 'GLO'),
 ('Waste in incineration', None),
 ('Waste in inert landfill', None),
 ('Cathode loss', None),
 ('Dross', None),
 ('Dross for recycling', None),
 ('Cathode iron ingots waste', None),
 ('Mineral waste, from mining', None),
 ('Production waste, not inert', None),
 ('Waste in bioactive landfill', None),
 ('copper production, primary', 'RER'),
 ('polyurethane production, flexible foam', 'RER'),
 ('glued laminated timber production, for outdoor use', 'RER'),
 ('market for electricity, medium voltage', 'RoW'),
 ('Zinc waste', None),
 ('Oil waste', None),
 ('Steel and iron (waste treatment) {GLO}| recycling of steel and iron | Alloc Rec, S',
  None),
 ('Aluminium (waste treatment) {GLO}| recycling of aluminium | Alloc Rec, S',
  None),
 ('PVC (waste treatment) {GLO}| recycling of PVC | Alloc Rec, S', None),
 ('Mixed plastics (waste treatment) {GLO}| recycling of mixed plastics | Alloc Rec, S',
  None),
 ('market for nitrogen fertiliser, as N', 'GLO'),
 ('market 

# Biosphere

In [ ]:
# del bd.databases["Agribalyse biosphere"]
# bd.Database("Agribalyse biosphere").register()
# ag.add_unlinked_flows_to_biosphere_database("Agribalyse biosphere")